# Interactive Tax Policy Analysis

このノートブックでは、インタラクティブなウィジェットを使用して、税制変更の影響をリアルタイムで分析できます。

In [ ]:
# 必要なライブラリのインポート
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 日本語フォント設定（最優先で実行）
print("=== 日本語フォント設定開始 ===")

# matplotlibのフォントキャッシュをクリア
import matplotlib.font_manager as fm
fm.fontManager.__init__()

# 直接的なフォント設定
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Hiragino Kaku Gothic Pro', 'Yu Gothic', 'DejaVu Sans', 'Arial']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print(f"✅ 日本語フォント設定完了")
print(f"現在のフォント: {plt.rcParams['font.sans-serif'][:3]}")

# プロジェクトのルートディレクトリを設定
if os.path.basename(os.getcwd()) == 'notebooks':
    project_root = os.path.abspath('..')
    os.chdir(project_root)
elif 'JapanTaxSimulator' in os.getcwd():
    while not os.path.exists('config/parameters.json') and os.getcwd() != '/':
        if 'JapanTaxSimulator' in os.getcwd():
            project_root = os.getcwd()
            break
        os.chdir('..')
    project_root = os.getcwd()
else:
    project_root = os.path.abspath('..')
    os.chdir(project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    print("⚠️ Plotlyが利用できません。基本的なMatplotlibプロットを使用します。")
    PLOTLY_AVAILABLE = False

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    print("⚠️ ipywidgetsが利用できません。静的な分析のみ実行します。")
    WIDGETS_AVAILABLE = False

# DSGEモデルのインポート
from src.dsge_model import DSGEModel, ModelParameters, load_model
from src.tax_simulator import EnhancedTaxSimulator, TaxReform

# スタイル設定
plt.style.use('seaborn-v0_8-darkgrid')

# スタイル設定後に日本語フォントを再設定（seabornがリセットする可能性があるため）
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Hiragino Kaku Gothic Pro', 'Yu Gothic', 'DejaVu Sans', 'Arial']
plt.rcParams['axes.unicode_minus'] = False
print("🔄 seaborn設定後のフォント再適用完了")

## 1. モデルの初期化

In [ ]:
# モデルの読み込みと初期化
try:
    model = load_model('config/parameters.json')  # パス修正
    
    # 定常状態の計算を確実に実行
    steady_state = model.compute_steady_state()
    
    # 定常状態が正常に計算されたかチェック
    if steady_state is None:
        print("❌ 定常状態の計算に失敗しました")
        raise Exception("定常状態の計算に失敗")
    
    print("✅ モデルと定常状態の計算に成功しました")
    
    # 税制シミュレータの初期化
    simulator = EnhancedTaxSimulator(model)
    print("✅ 税制シミュレータの初期化に成功しました")
    
    # ベースラインの定常状態を取得
    baseline_ss = model.steady_state.to_dict()

    print(f"\nベースライン税率:")
    print(f"  消費税: {model.params.tau_c:.1%}")
    print(f"  所得税: {model.params.tau_l:.1%}")
    print(f"  資本所得税: {model.params.tau_k:.1%}")
    print(f"  法人税: {model.params.tau_f:.1%}")
    
    print(f"\n主要経済変数の定常状態:")
    for var in ['Y', 'C', 'I', 'L']:
        if var in baseline_ss:
            print(f"  {var}: {baseline_ss[var]:.3f}")

except Exception as e:
    print(f"❌ 初期化に失敗しました: {e}")
    print("\n代替的な初期化を試みます...")
    
    # エラー発生時の代替措置
    import traceback
    traceback.print_exc()
    
    # シミュレータをNoneに設定してエラーを回避
    simulator = None
    baseline_ss = None
    model = None

## 2. インタラクティブ税制シミュレーター

In [ ]:
# インタラクティブ税制シミュレーター（ウィジェット利用可能時）
if WIDGETS_AVAILABLE and simulator is not None:
    # Safe percentage change function
    def safe_percentage_change(new_val, old_val, default=0.0):
        """Safe percentage change calculation that handles zero denominators"""
        if abs(old_val) < 1e-10:
            return default if abs(new_val) >= 1e-10 else 0.0
        return (new_val - old_val) / old_val * 100
    
    # ウィジェットの作成
    tau_c_slider = widgets.FloatSlider(
        value=0.10,
        min=0.0,
        max=0.30,
        step=0.01,
        description='消費税率:',
        continuous_update=False
    )

    tau_l_slider = widgets.FloatSlider(
        value=0.20,
        min=0.0,
        max=0.50,
        step=0.01,
        description='所得税率:',
        continuous_update=False
    )

    tau_k_slider = widgets.FloatSlider(
        value=0.25,
        min=0.0,
        max=0.50,
        step=0.01,
        description='資本所得税率:',
        continuous_update=False
    )

    tau_f_slider = widgets.FloatSlider(
        value=0.30,
        min=0.0,
        max=0.50,
        step=0.01,
        description='法人税率:',
        continuous_update=False
    )

    implementation_dropdown = widgets.Dropdown(
        options=['permanent', 'temporary', 'phased'],
        value='permanent',
        description='実施方法:'
    )

    periods_slider = widgets.IntSlider(
        value=40,
        min=20,
        max=100,
        step=10,
        description='期間:',
        continuous_update=False
    )

    # 結果表示用の出力エリア
    output = widgets.Output()

    # シミュレーション実行関数
    def run_simulation(tau_c, tau_l, tau_k, tau_f, implementation, periods):
        with output:
            output.clear_output(wait=True)
            
            # 税制改革の設定
            reform = TaxReform(
                name="カスタム税制改革",
                tau_c=tau_c,
                tau_l=tau_l,
                tau_k=tau_k,
                tau_f=tau_f,
                implementation=implementation
            )
            
            try:
                # シミュレーション実行
                print("シミュレーション実行中...")
                results = simulator.simulate_reform(reform, periods=periods)
                print("✅ シミュレーション完了")
                
                # 基本的な結果表示
                print(f"\n=== 税制改革の影響 ===")
                print(f"改革名: {results.name}")
                print(f"厚生変化: {results.welfare_change:.2f}%")
                print(f"移行期間: {results.transition_periods} 四半期")
                
                # 定常状態への影響を表示（ゼロ除算対策付き）
                print("\n=== 定常状態への影響（%変化） ===")
                new_ss = results.steady_state_reform
                
                for var in ['Y', 'C', 'I', 'L']:
                    if hasattr(new_ss, var) and var in baseline_ss:
                        old_val = baseline_ss[var]
                        new_val = getattr(new_ss, var)
                        
                        # ゼロ除算の安全な処理
                        change = safe_percentage_change(new_val, old_val)
                        print(f"  {var}: {change:+.2f}%")
                
                # 簡単なMatplotlibプロット
                fig, axes = plt.subplots(2, 2, figsize=(12, 8))
                variables = ['Y', 'C', 'I', 'L']
                positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
                
                for var, (row, col) in zip(variables, positions):
                    if var in results.reform_path.columns:
                        ax = axes[row, col]
                        ax.plot(results.baseline_path.index, results.baseline_path[var], 
                               'b--', label='ベースライン', alpha=0.7)
                        ax.plot(results.reform_path.index, results.reform_path[var], 
                               'r-', label='改革後', linewidth=2)
                        ax.set_title(var)
                        ax.set_xlabel('期間')
                        ax.legend()
                        ax.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
                    
            except Exception as e:
                print(f"❌ シミュレーションエラー: {e}")
                import traceback
                traceback.print_exc()

    # インタラクティブウィジェットの設定
    interactive_plot = widgets.interactive(
        run_simulation,
        tau_c=tau_c_slider,
        tau_l=tau_l_slider,
        tau_k=tau_k_slider,
        tau_f=tau_f_slider,
        implementation=implementation_dropdown,
        periods=periods_slider
    )

    # ウィジェットの表示
    display(interactive_plot, output)

else:
    print("⚠️ ウィジェットまたはシミュレータが利用できません。")
    print("代替として静的な分析を実行します。")
    
    if simulator is not None:
        # Safe percentage change function for static analysis
        def safe_percentage_change(new_val, old_val, default=0.0):
            """Safe percentage change calculation that handles zero denominators"""
            if abs(old_val) < 1e-10:
                return default if abs(new_val) >= 1e-10 else 0.0
            return (new_val - old_val) / old_val * 100
        
        # 静的なシミュレーション例
        print("\n=== 静的シミュレーション例 ===")
        
        # 消費税5%引き上げのシミュレーション
        reform_example = TaxReform(
            name="消費税5%引き上げ",
            tau_c=0.15,
            implementation='permanent'
        )
        
        try:
            results = simulator.simulate_reform(reform_example, periods=40)
            print(f"✅ シミュレーション完了: {results.name}")
            print(f"厚生変化: {results.welfare_change:.2f}%")
            
            # 基本的なプロット
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            variables = ['Y', 'C', 'I', 'L']
            positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
            
            for var, (row, col) in zip(variables, positions):
                if var in results.reform_path.columns:
                    ax = axes[row, col]
                    ax.plot(results.baseline_path.index, results.baseline_path[var], 
                           'b--', label='ベースライン', alpha=0.7)
                    ax.plot(results.reform_path.index, results.reform_path[var], 
                           'r-', label='改革後', linewidth=2)
                    ax.set_title(var)
                    ax.set_xlabel('期間')
                    ax.legend()
                    ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
                
        except Exception as e:
            print(f"❌ 静的シミュレーションエラー: {e}")
    else:
        print("❌ シミュレータが初期化されていません。")

## 3. 税収最大化分析

In [ ]:
# ラッファー曲線の描画
def compute_laffer_curve(tax_type='consumption', tax_rates=None):
    if tax_rates is None:
        tax_rates = np.linspace(0, 0.5, 21)
    
    revenues = []
    gdp_levels = []
    
    for rate in tax_rates:
        # パラメータ設定
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        # 税率を変更
        if tax_type == 'consumption':
            test_params.tau_c = rate
        elif tax_type == 'income':
            test_params.tau_l = rate
        elif tax_type == 'capital':
            test_params.tau_k = rate
        elif tax_type == 'corporate':
            test_params.tau_f = rate
        
        # モデル計算
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            revenues.append(ss.T_total_revenue)
            gdp_levels.append(ss.Y)
        except:
            revenues.append(np.nan)
            gdp_levels.append(np.nan)
    
    return tax_rates, revenues, gdp_levels

# Safe utilities for Plotly
def safe_max_revenue_calculation(rates, revenues):
    """Safely find the maximum revenue point"""
    valid_indices = ~np.isnan(revenues)
    if not np.any(valid_indices):
        return rates[0], revenues[0] if revenues else 0.0
    
    valid_revenues = np.array(revenues)[valid_indices]
    valid_rates = np.array(rates)[valid_indices]
    
    if len(valid_revenues) == 0:
        return rates[0], 0.0
    
    max_idx = np.argmax(valid_revenues)
    return valid_rates[max_idx], valid_revenues[max_idx]

# インタラクティブラッファー曲線
if WIDGETS_AVAILABLE and simulator is not None:
    tax_type_dropdown = widgets.Dropdown(
        options=['consumption', 'income', 'capital', 'corporate'],
        value='consumption',
        description='税種別:'
    )

    laffer_output = widgets.Output()

    def plot_laffer_curve(tax_type):
        with laffer_output:
            laffer_output.clear_output(wait=True)
            
            print(f"{tax_type}税のラッファー曲線を計算中...")
            try:
                rates, revenues, gdp_levels = compute_laffer_curve(tax_type)
                
                if PLOTLY_AVAILABLE:
                    # プロット作成
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=('税収', 'GDP')
                    )
                    
                    # 税収のプロット
                    fig.add_trace(
                        go.Scatter(
                            x=rates * 100,
                            y=revenues,
                            mode='lines+markers',
                            name='税収'
                        ),
                        row=1, col=1
                    )
                    
                    # GDPのプロット
                    fig.add_trace(
                        go.Scatter(
                            x=rates * 100,
                            y=gdp_levels,
                            mode='lines+markers',
                            name='GDP'
                        ),
                        row=1, col=2
                    )
                    
                    fig.update_xaxes(title_text="税率 (%)", row=1, col=1)
                    fig.update_xaxes(title_text="税率 (%)", row=1, col=2)
                    fig.update_yaxes(title_text="税収", row=1, col=1)
                    fig.update_yaxes(title_text="GDP", row=1, col=2)
                    
                    fig.update_layout(
                        height=400,
                        title_text=f"{tax_type}税のラッファー曲線",
                        showlegend=False
                    )
                    
                    fig.show()
                else:
                    # Matplotlib fallback
                    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
                    
                    axes[0].plot(rates * 100, revenues, 'o-')
                    axes[0].set_xlabel('税率 (%)')
                    axes[0].set_ylabel('税収')
                    axes[0].set_title('税収')
                    axes[0].grid(True)
                    
                    axes[1].plot(rates * 100, gdp_levels, 'o-')
                    axes[1].set_xlabel('税率 (%)')
                    axes[1].set_ylabel('GDP')
                    axes[1].set_title('GDP')
                    axes[1].grid(True)
                    
                    plt.suptitle(f"{tax_type}税のラッファー曲線")
                    plt.tight_layout()
                    plt.show()
                
                # 最大税収をもたらす税率を表示（ゼロ除算対策）
                optimal_rate, max_revenue = safe_max_revenue_calculation(rates, revenues)
                
                print(f"\n最大税収をもたらす税率: {optimal_rate:.1%}")
                print(f"最大税収: {max_revenue:.3f}")
                
            except Exception as e:
                print(f"❌ ラッファー曲線の計算に失敗: {e}")
                import traceback
                traceback.print_exc()

    interactive_laffer = widgets.interactive(
        plot_laffer_curve,
        tax_type=tax_type_dropdown
    )

    display(interactive_laffer, laffer_output)
else:
    print("⚠️ ウィジェットが利用できません。基本的なラッファー曲線を表示します。")
    
    if simulator is not None and model is not None:
        try:
            rates, revenues, gdp_levels = compute_laffer_curve('consumption')
            
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            
            axes[0].plot(rates * 100, revenues, 'o-')
            axes[0].set_xlabel('税率 (%)')
            axes[0].set_ylabel('税収')
            axes[0].set_title('消費税のラッファー曲線 - 税収')
            axes[0].grid(True)
            
            axes[1].plot(rates * 100, gdp_levels, 'o-')
            axes[1].set_xlabel('税率 (%)')
            axes[1].set_ylabel('GDP')
            axes[1].set_title('消費税のラッファー曲線 - GDP')
            axes[1].grid(True)
            
            plt.tight_layout()
            plt.show()
            
            optimal_rate, max_revenue = safe_max_revenue_calculation(rates, revenues)
            print(f"最大税収をもたらす消費税率: {optimal_rate:.1%}")
            print(f"最大税収: {max_revenue:.3f}")
            
        except Exception as e:
            print(f"❌ ラッファー曲線の計算に失敗: {e}")

## 4. 動的経路の3D可視化

In [ ]:
# 3D動的経路の可視化
def plot_3d_dynamics():
    if not PLOTLY_AVAILABLE or simulator is None:
        print("⚠️ 3D可視化にはPlotlyとシミュレータが必要です。")
        return
    
    # 複数の税制シナリオを比較
    scenarios = [
        {'name': 'ベースライン', 'tau_c': 0.10, 'tau_l': 0.20},
        {'name': '消費税増税', 'tau_c': 0.15, 'tau_l': 0.20},
        {'name': '所得税減税', 'tau_c': 0.10, 'tau_l': 0.15},
        {'name': '複合改革', 'tau_c': 0.15, 'tau_l': 0.15}
    ]
    
    fig = go.Figure()
    
    for scenario in scenarios:
        reform = TaxReform(
            name=scenario['name'],
            tau_c=scenario['tau_c'],
            tau_l=scenario['tau_l'],
            implementation='permanent'
        )
        
        try:
            results = simulator.simulate_reform(reform, periods=40)
            
            # Check if reform_path exists
            if hasattr(results, 'reform_path'):
                path_data = results.reform_path
            elif hasattr(results, 'paths'):
                path_data = results.paths
            else:
                print(f"⚠️ {scenario['name']}: 経路データが見つかりません")
                continue
            
            # 3次元プロット（GDP, 消費, 投資）
            fig.add_trace(go.Scatter3d(
                x=path_data['Y'],
                y=path_data['C'],
                z=path_data['I'],
                mode='lines+markers',
                name=scenario['name'],
                marker=dict(size=3),
                line=dict(width=3)
            ))
        except Exception as e:
            print(f"⚠️ {scenario['name']}のシミュレーションに失敗: {e}")
    
    fig.update_layout(
        title='税制改革の動的経路（3D）',
        scene=dict(
            xaxis_title='GDP',
            yaxis_title='消費',
            zaxis_title='投資'
        ),
        height=700
    )
    
    fig.show()

# 3Dプロットの実行
try:
    plot_3d_dynamics()
except Exception as e:
    print(f"❌ 3Dプロットの実行に失敗: {e}")
    print("代替として基本的なプロットを表示します")

## 5. 税制改革の最適化

In [ ]:
# 税制最適化問題
def optimize_tax_mix(objective='welfare', constraint='revenue_neutral'):
    """
    目的関数を最大化する税制ミックスを探索（ZeroDivisionError対策付き）
    """
    from scipy.optimize import minimize
    
    # ベースラインの税収を取得（ゼロチェック付き）
    try:
        baseline_revenue = model.steady_state.T_total_revenue
        if baseline_revenue <= 0:
            print("⚠️ ベースライン税収が不正な値です。最適化をスキップします。")
            return None
    except AttributeError:
        print("⚠️ ベースライン税収データが利用できません。")
        return None
    
    def safe_divide_for_optimization(numerator, denominator, default=0.0):
        """最適化用の安全な除算"""
        if abs(denominator) < 1e-10:
            return default
        return numerator / denominator
    
    def objective_function(x):
        tau_c, tau_l = x
        
        # 極端な値をクリップ
        tau_c = np.clip(tau_c, 0.001, 0.299)
        tau_l = np.clip(tau_l, 0.001, 0.499)
        
        # パラメータ設定
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        test_params.tau_c = tau_c
        test_params.tau_l = tau_l
        
        # モデル計算
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            
            # 基本的な妥当性チェック
            if ss.Y <= 0 or ss.C <= 0 or ss.L <= 0:
                return 1e10  # 不正な解にペナルティ
            
            if objective == 'welfare':
                # 厚生（消費）を最大化
                return -ss.C  # 最小化問題なので符号を反転
            elif objective == 'gdp':
                return -ss.Y
            elif objective == 'employment':
                return -ss.L
        except Exception as e:
            # 計算失敗時のペナルティ
            return 1e10
    
    def constraint_function(x):
        tau_c, tau_l = x
        
        # 極端な値をクリップ
        tau_c = np.clip(tau_c, 0.001, 0.299)
        tau_l = np.clip(tau_l, 0.001, 0.499)
        
        test_params = ModelParameters()
        for attr in dir(model.params):
            if not attr.startswith('_'):
                setattr(test_params, attr, getattr(model.params, attr))
        
        test_params.tau_c = tau_c
        test_params.tau_l = tau_l
        
        test_model = DSGEModel(test_params)
        try:
            ss = test_model.compute_steady_state()
            if constraint == 'revenue_neutral':
                return ss.T_total_revenue - baseline_revenue
        except Exception as e:
            return -1e10
    
    # 最適化実行
    x0 = [model.params.tau_c, model.params.tau_l]
    bounds = [(0.001, 0.299), (0.001, 0.499)]  # ゼロを避ける
    
    try:
        if constraint:
            constraints = {'type': 'eq', 'fun': constraint_function}
            result = minimize(objective_function, x0, bounds=bounds, constraints=constraints, 
                            method='SLSQP', options={'maxiter': 50, 'ftol': 1e-6})
        else:
            result = minimize(objective_function, x0, bounds=bounds, method='L-BFGS-B',
                            options={'maxiter': 50, 'ftol': 1e-6})
        
        return result
    except Exception as e:
        print(f"⚠️ 最適化の実行中にエラーが発生しました: {e}")
        return None

# 最適化結果の表示
if simulator is not None and model is not None:
    print("税制最適化の実行...")
    try:
        result = optimize_tax_mix(objective='welfare', constraint='revenue_neutral')

        if result is not None and result.success:
            optimal_tau_c, optimal_tau_l = result.x
            print(f"\n✅ 最適な税制ミックス（厚生最大化、税収中立）:")
            print(f"  消費税率: {optimal_tau_c:.1%}")
            print(f"  所得税率: {optimal_tau_l:.1%}")
            print(f"\n現行税制との比較:")
            print(f"  消費税: {model.params.tau_c:.1%} → {optimal_tau_c:.1%}")
            print(f"  所得税: {model.params.tau_l:.1%} → {optimal_tau_l:.1%}")
        elif result is not None:
            print(f"❌ 最適化に失敗しました: {result.message}")
        else:
            print("❌ 最適化を実行できませんでした。")
    except Exception as e:
        print(f"❌ 最適化でエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ 必要なモデルデータが利用できません。")

## 6. 感応度分析ヒートマップ

In [ ]:
# 2つの税率の組み合わせによる影響をヒートマップで表示
def create_sensitivity_heatmap(variable='Y'):
    """Create sensitivity heatmap with ZeroDivisionError protection"""
    tau_c_range = np.linspace(0.05, 0.20, 10)
    tau_l_range = np.linspace(0.10, 0.30, 10)
    
    results = np.zeros((len(tau_l_range), len(tau_c_range)))
    
    # Safe percentage change function
    def safe_heatmap_calc(new_val, baseline_val):
        if abs(baseline_val) < 1e-10:
            return 0.0 if abs(new_val) < 1e-10 else 100.0  # 100% for visual distinction
        return (new_val - baseline_val) / baseline_val * 100
    
    for i, tau_l in enumerate(tau_l_range):
        for j, tau_c in enumerate(tau_c_range):
            test_params = ModelParameters()
            for attr in dir(model.params):
                if not attr.startswith('_'):
                    setattr(test_params, attr, getattr(model.params, attr))
            
            test_params.tau_c = tau_c
            test_params.tau_l = tau_l
            
            test_model = DSGEModel(test_params)
            try:
                ss = test_model.compute_steady_state()
                baseline_val = getattr(model.steady_state, variable)
                new_val = getattr(ss, variable)
                
                # Safe percentage calculation
                results[i, j] = safe_heatmap_calc(new_val, baseline_val)
                
            except Exception as e:
                print(f"⚠️ Error at tau_c={tau_c:.2%}, tau_l={tau_l:.2%}: {e}")
                results[i, j] = np.nan
    
    if PLOTLY_AVAILABLE:
        # ヒートマップの作成
        fig = go.Figure(data=go.Heatmap(
            z=results,
            x=tau_c_range * 100,
            y=tau_l_range * 100,
            colorscale='RdBu',
            zmid=0,
            text=np.round(results, 1),
            texttemplate='%{text}%',
            textfont={"size": 10},
            colorbar=dict(title="変化率 (%)")
        ))
        
        fig.update_layout(
            title=f'{variable}の感応度分析',
            xaxis_title='消費税率 (%)',
            yaxis_title='所得税率 (%)',
            width=700,
            height=600
        )
        
        # 現行税制の位置をマーク
        fig.add_scatter(
            x=[model.params.tau_c * 100],
            y=[model.params.tau_l * 100],
            mode='markers',
            marker=dict(size=15, color='red', symbol='x'),
            name='現行税制'
        )
        
        fig.show()
    else:
        # Matplotlib fallback
        fig, ax = plt.subplots(figsize=(10, 8))
        
        im = ax.imshow(results, cmap='RdBu', aspect='auto', 
                      extent=[tau_c_range[0]*100, tau_c_range[-1]*100,
                             tau_l_range[0]*100, tau_l_range[-1]*100],
                      vmin=-np.nanmax(np.abs(results)), vmax=np.nanmax(np.abs(results)))
        
        ax.set_xlabel('消費税率 (%)')
        ax.set_ylabel('所得税率 (%)')
        ax.set_title(f'{variable}の感応度分析')
        
        # Current tax system marker
        ax.plot(model.params.tau_c * 100, model.params.tau_l * 100, 
               'rx', markersize=15, markeredgewidth=3, label='現行税制')
        
        plt.colorbar(im, ax=ax, label='変化率 (%)')
        plt.legend()
        plt.tight_layout()
        plt.show()

# ヒートマップの表示
if WIDGETS_AVAILABLE and simulator is not None and model is not None:
    variable_dropdown = widgets.Dropdown(
        options=['Y', 'C', 'I', 'L', 'T_total_revenue'],
        value='Y',
        description='変数:'
    )

    interactive_heatmap = widgets.interactive(
        create_sensitivity_heatmap,
        variable=variable_dropdown
    )

    display(interactive_heatmap)
else:
    print("⚠️ ウィジェットまたは必要なデータが利用できません。")
    
    if model is not None:
        try:
            print("基本的な感応度分析を実行中...")
            create_sensitivity_heatmap('Y')
        except Exception as e:
            print(f"❌ 感応度分析に失敗: {e}")
    else:
        print("❌ モデルが初期化されていません。")

## まとめ

このインタラクティブノートブックでは、以下の分析ツールを提供しました：

1. **リアルタイム税制シミュレーター**: スライダーで税率を調整し、即座に経済への影響を確認
2. **ラッファー曲線分析**: 各税種別の税収最大化ポイントを特定
3. **3D動的経路分析**: 複数シナリオの経済変数の動的経路を3次元で可視化
4. **税制最適化**: 厚生や成長を最大化する最適な税制ミックスを探索
5. **感応度分析**: 2つの税率の組み合わせによる影響をヒートマップで表示

これらのツールを活用することで、政策立案者は税制改革の影響を多角的に分析し、より良い政策決定を行うことができます。